Boilerplate

In [11]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19` 
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import $ivy.$                                                  

import $ivy.$                                                          

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

# Sequential Logic

_In the previous section, we built the combinational part of the LFSR - that is, given the simultaneous output of all the shift register bits, generate the bit to feed back in. In this section, we will build the sequential part, the shift registers themselves._

The basic stateful element in Chisel is the shift register, or `Reg`, which stores its current value through a clock cycle, and can optionally be updated with a new value which becomes visible at the next clock edge.

A shift register can be constructed given a data type. For instance, a 4-bit wide shift register without initialization:
```scala
val myReg = Reg(UInt(4.W))
```

or with initialization (to 2 decimal):
```scala
val myReg = RegInit(UInt(4.W), 2.U)
```

Its value can be updated with the `:=` operator. For example, this increments the register's value:
```scala
myReg := myReg + 1.U
```

Note that when read, it gives its current (stored) value, but the assignment doesn't take effect until the next clock edge.

Below is an example module which demonstrates a single-bit-wide shift register. Note how it initializes at zero, and afterwards the output is a single-cycle delayed version of the input.

In [10]:
// Module containing the register
class MyRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(Bool())
  })
  val reg = RegInit(Bool(), false.B)
  reg := io.in
  io.out := reg
  printf("in=%d, out=%d\n", io.in, io.out)
}

// Testvector
class MyRegisterTester(c: MyRegister) extends PeekPokeTester(c) {
  poke(c.io.in, 0)
  step(1)
  poke(c.io.in, 1)
  step(1)
  poke(c.io.in, 1)
  step(1)
  poke(c.io.in, 0)
  step(1)
}

// Driver run invocation
Driver(() => new MyRegister, "firrtl") {
  c => new MyRegisterTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 15.0 ms
Total FIRRTL Compile Time: 8.7 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501544648128
in=0, out=0
in=1, out=1
in=1, out=1
in=0, out=0
test cmd9WrapperHelperMyRegister Success: 0 tests passed in 9 cycles taking 0.003435 seconds
[info] [0.002] RAN 4 CYCLES PASSED


defined class MyRegister
defined class MyRegisterTester
res9_2: Boolean = true

Given that, now build a module that implements a shift register for your LFSR. Specifically:
- Each element is a single bit wide (of `Bool` type).
- Has 4 stages.
- Takes a single input bit, which is the next value into the shift register.
- Outputs the parallel output of the shift register, with the MSbit being the last element of the shift register and the LSbit being the first element of the shift register. `Cat` may come in handy.
- The output initializes at `b0001`.
- Shifts each clock cycle (no enable signal).

A basic Module skeleton, testvector, and Driver invocation is provided below. The first register has been provided for you.

In [13]:
class MyShiftRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt(4.W))
  })
  val s1 = RegInit(Bool(), true.B)
    
  // YOUR CODE HERE
  // don't forget to update the first register!
    
  s1 := io.in
  val s2 = RegInit(Bool(), false.B)
  s2 := s1
  val s3 = RegInit(Bool(), false.B)
  s3 := s2
  val s4 = RegInit(Bool(), false.B)
  s4 := s3
    
  io.out := Cat(s4, s3, s2, s1)
}

defined class MyShiftRegister

In [18]:
class MyShiftRegisterTester(c: MyShiftRegister) extends PeekPokeTester(c) {
  expect(c.io.out, 1)  // b0001
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 2)  // b0010
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 5)  // b0101
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 11)  // b1011
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 6)  // b0110
}

Driver(() => new MyShiftRegister, "firrtl") {
  c => new MyShiftRegisterTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 14.2 ms
Total FIRRTL Compile Time: 9.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501545250574
test cmd12WrapperHelperMyShiftRegister Success: 5 tests passed in 9 cycles taking 0.004067 seconds
[info] [0.003] RAN 4 CYCLES PASSED


defined class MyShiftRegisterTester
res17_1: Boolean = true